## Copula

### 1.  A short introduction to Copula

Copula is a multivariate probability distribution with special characteristics(for which the marginal probability distribution of each variable is uniform). Usually, there are some copula families like Gaussian copula, Archimedean copulas and so on, with similar forms of generator functions which are used to generate its pdf, cdf... They all have parameters in their generator funtion that control the strength of dependence.

### 2. Code of archimedean copula

#### 2.1 Python packages needed

We need three python packages: Numpy, Sympy, Scipy.
- Numpy: we use 'ndarray' as our input and output. So Numpy is a fundamental package that provides with its data structures.
- Sympy: we use sympy to generate high order of differentiation to calculate the pdf. We had planned to scipy to get differebtaition, however, after testing the result from scipy in high order of differentiation, we found there were great error in its results. By contrast sympy has accurate results, so we choose it.
- Scipy: we use scipy to integrate to get the value of tau. Scipy can be replaced by sympy if needed. We can discuss about which to use.

In [3]:
# ---- import packages -----
import numpy as np
import sympy as sp
from scipy import integrate


#### 2.2 Archimedean Copula

Archimedean copulas are an associative class of copulas. Most common Archimedean copulas admit an explicit formula. In practice, Archimedean copulas are popular because they allow modeling dependence in arbitrarily high dimensions with only one parameter, governing the strength of dependence.

Due to the differences between Archimedean Copula and Ellip Copula and functions we may use, we plan to divide the whole copula class to two parts, which means Archimedean Copula doesn't inherit from any class. Archimedean and Ellip are two seperated classes.

![](https://github.com/codeandworld/photo/blob/master/copula_pic.JPG?raw=true)

In [4]:
# ---- Aechemidean Coupla -----
class ArcmCopula(object):
    """
    Note to reader: Each bivariate copula should be its own subclass that implements the following methods.
    """

    def __init__(self):
        pass

    def pdf(self):
        """
        Evaluate the probability distribution function (pdf) at a point (u,v) for a parameter theta
        """
        pass

    def cdf(self):
        """
        Evaluate the cumulative distribution function (cdf) at a point (u,v) for a parameter theta
        """
        pass

    def rvs(self):
        """
        Generate a simulation for the copula given a specified parameter theta.
        """
        pass

    def rho(self):
        """
        Calculate Spearman's rho for the copula given a specified parameter theta.
        """
        pass

    def tau(self):
        """
        Calculate Kendall's tau for the copula given a specified parameter theta.
        """
        pass

#### 2.3 Joe Copula 

Joe copula is one of the Archimedean copulas. The Class Joe Copula includes four functions, pdf, cdf, rvs and tau. All can be calsulated through the generator function.

- cdf: cdf can be calculated using the combination of generator funtion and inverse generator function.
- pdf: pdf can be calculated using the combination of generator funtion and the high order differentiation of inverse generator function. Here, we firstly generate the high order differentiation by Sympy. Then combine all the formula together.
- rvs: rvs can be generated using Monte Carlo simulation. We generate random numbers and put the right one in the array.
- tau: tau can be calculated using generator funtion.

In [4]:
# ---- Joe Coupla -----
class JoeCopula(object): # object waiting to change 
    """
    An joe_copula with one parameter theta.
    Get pdf, cdf, rvs, tau given 2D-ndarray,
    seeing each row as a point in distribution.
    
    Parameters
    ----------
        x: 2D-ndarray;
           number in x should be between (0,1);
    theta: float;
           number should larger than one
    
    Examples
    -----------
    x = np.array([[0.1,0.2,0.3],[0.2,0.3,0.4]])
    myjoe = JoeCopula()
    myjoe.pdf(x,1.2)
    """
    def __init__(self):
        pass
    
    def cdf(self, x, theta):
        """
        genarator function : f(x)
        inverse genarator function : g(x)
        X = (x1, x2, ... ,xn)
        cdf(X) = g(f(x1)+f(x2)+...+f(xn))
        """
        term = np.sum(-np.log(1.0-(1.0-x)**theta),axis=-1)
        return 1.0-(1.0-np.exp(-term))**(1.0/theta)
    
    def pdf(self, x, theta):
        """
        genarator function : f(x)
        inverse genarator function : g(x)
        X = (x1, x2, ... ,xn)
        pdf(X) = g^{(-n)}(f(x1)+f(x2)+...+f(xn))f'(x1)f'(x2)...f'(xn)
        """
        dim = x.shape[-1]
        term = np.sum(-np.log(1.0-(1.0-x)**theta),axis=-1)
        
        # get the differentiation of the genarator function and inverse genarator function
        t = sp.Symbol('t')
        f1 = 1.0-(1.0-sp.exp(-t))**(1.0/theta)
        for i in range(dim):
            f1 = sp.diff(f1,t)
        f2 = -sp.log(1.0-(1.0-t)**theta)                       
        f2 = sp.diff(f2,t)
        
        # get ufunc of the differentiation equation
        u1 = lambda z: f1.subs(t,z)
        u1 = np.frompyfunc(u1,1,1)
        u2 = lambda z: f2.subs(t,z)
        u2 = np.frompyfunc(u2,1,1)
        return u1(term)*np.prod(u2(x),axis=-1)
    
    def rvs(self, n, dim, theta):
        """
        Monte Carlo simulation using JoeCopula.pdf(self, k, theta)
        """
        i = 0
        while(i<n):
            k = np.random.random_sample(dim)
            if theta*np.random.random_sample(1)< JoeCopula.pdf(self, k, theta):
                if i == 0:
                    sample = k
                else:
                    sample = np.vstack((sample,k))
                i = i+1
        return sample

    def tau(self, x, theta):
        """
        genarator function : f(t)
        $$tau(x)=1+\int^1_0 \frac{f(t)}{f'(t)}dt$$
        """
        f = lambda t: np.log(1.0-(1.0-t)**theta)/(theta*(1.0-t)**(theta-1.0)/(1.0-(1.0-t)**theta))# f = (-log(1-(1-t)^theta)) / (theta*(1-t)^theta/(1-(1-t)^theta))
        return 1.0+4.0*integrate.quad(f, 0, 1.0)[0]
       

### 3. Result testing comparing with R

##### 3.1 R and Python code for testing pdf 


In [5]:
x = np.array([[0.1,0.2,0.3],[0.2,0.3,0.4]])
myjoe = JoeCopula()
myjoe.pdf(x, 1.1)

array([1.13307613035283, 1.09741762777262], dtype=object)

In [13]:
library(copula)
joe.cop <- joeCopula(1.1, dim=3)
x <- dCopula(c(0.1,0.2,0.3),joe.cop)
x

[1] 1.133076

##### 3.2 R and Python code for testing cdf 


In [7]:
myjoe.cdf(x, 1.1)

array([ 0.0070295 ,  0.02764251])

In [14]:
joe.cop <- joeCopula(1.1, dim=3)
pCopula(c(0.1,0.2,0.3),joe.cop)

[1] 0.007029504

##### 3.3 R and Python code for testing tau 


In [6]:
myjoe.tau(x, 1.1)

0.05439832056368421

In [15]:
joe.cop <- joeCopula(1.1, dim=3)
tau(joe.cop)

[1] 0.05439832

##### 3.4 Python code for testing rvs

In [9]:
myjoe.rvs(n=3,dim=4,theta=1.2)

array([[ 0.89689821,  0.20786522,  0.14268674,  0.43908519],
       [ 0.67671012,  0.70005462,  0.70013851,  0.17352006],
       [ 0.89842734,  0.90756495,  0.3997748 ,  0.52008395]])

### 4. Problems facing

Although the functions have been done, but there are still some problems we may need to discuss about.
- First is about the design of the class. We plan to write two seperated class, one is Gaussian Copual, the other is Archimedean copula.Joe can extend from Archimedean copula. Gaussian and Archimedean have lots of differences not only in their generation, but also in their functions. We also refer to matlab and R code of copula, they both have two seperated classes. What is your point about that?
- Second is about rvs. Now we use Monte Carlo simulation to calculate random numbers. And we have to calculate them one by one. In R, it seems that there are some functions can be used to generate random numbers. However, i havent seen any theory about generating random numbers in my book. Do you know how to generate them?
- For integration, i tested both the scipy and sympy. Considering the precision of their results, sympy must be a better choice. However, we have to use one more packages. Totally, we should import three packages. What do you think about that reducing the dependence or having precise answers? 
